In [ ]:
# pip install scikeras

In [ ]:
'''
 -----------------------------------------------------------
          Artificial Intelligence Workshop RUG
 -----------------------------------------------------------
            R.M. (Rolando) Gonzales Martinez
 -----------------------------------------------------------
 ~ ~ ~ ~ ~ ~ ~  Population forecasts with AI  ~ ~ ~ ~ ~ ~ ~
    Small area population forecasts with LSTM & GRU
   models and hyper-parameter fine-tuning with grid search
 -----------------------------------------------------------
'''
import pandas as pd
import os, random
import numpy as np
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# 1. Loading data
url = "https://raw.githubusercontent.com/rogon666/AI_workshop/refs/heads/main/02_databases/Berlinpopulation.csv"
df = pd.read_csv(url)

In [ ]:
# Grid search hypertuning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import product
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, GRU


# 2. TRAIN/TEST SPLIT
TEST_SIZE = 36
train_df = df.iloc[:-TEST_SIZE].reset_index(drop=True)
test_df  = df.iloc[-TEST_SIZE:].reset_index(drop=True)

# 3. SCALE POPULATION
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df['population'].values.reshape(-1,1))

# 4. MAKE SEQUENCES
SEQ_LEN = 10
def make_sequences(series, seq_len):
    X, y = [], []
    for i in range(seq_len, len(series)):
        X.append(series[i-seq_len:i, 0])
        y.append(series[i, 0])
    return np.array(X).reshape(-1, seq_len, 1), np.array(y)

x_train_seq, y_train_seq = make_sequences(train_scaled, SEQ_LEN)

# 5. GRID SEARCH w/ ONE OVERALL PROGRESS BAR
param_grid = {
    'cell_type':  ['LSTM', 'GRU'],
    'units':      [20, 50, 100],
    'batch_size': [1, 5],
    'epochs':     [5, ] # <--------------------------- fill here
}
param_list = list(product(
    param_grid['cell_type'],
    param_grid['units'],
    param_grid['batch_size'],
    param_grid['epochs']
))
tscv = TimeSeriesSplit(n_splits=3)

# track best for each cell type
best_score = {'LSTM': np.inf, 'GRU': np.inf}
best_params = {'LSTM': None,   'GRU': None}

print("Starting grid search:")
for cell_type, units, batch_size, epochs in tqdm(
    param_list,
    desc="Overall grid search",
    unit="config",
    ncols=80
):
    cv_scores = []
    for train_idx, val_idx in tscv.split(x_train_seq):
        X_tr, X_val = x_train_seq[train_idx], x_train_seq[val_idx]
        y_tr, y_val = y_train_seq[train_idx], y_train_seq[val_idx]

        # build model with Input layer
        model = Sequential([
            Input(shape=(SEQ_LEN,1)),
            (LSTM(units, return_sequences=True) if cell_type=='LSTM'
             else GRU(units, return_sequences=True)),
            (LSTM(units) if cell_type=='LSTM'
             else GRU(units)),
            Dense(1)
        ])
        model.compile(loss='mean_squared_error', optimizer='adam')

        # train silently
        model.fit(
            X_tr, y_tr,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            shuffle=False
        )

        preds = model.predict(X_val, verbose=0).flatten()
        cv_scores.append(mean_squared_error(y_val, preds))

    mean_cv = np.mean(cv_scores)
    if mean_cv < best_score[cell_type]:
        best_score[cell_type] = mean_cv
        best_params[cell_type] = {
            'units':      units,
            'batch_size': batch_size,
            'epochs':     epochs
        }

# report best for each
print("\nBest hyperparameters per model:")
for ct in ['LSTM','GRU']:
    bp = best_params[ct]
    print(f"{ct}: units={bp['units']}, batch_size={bp['batch_size']}, "
          f"epochs={bp['epochs']} → CV MSE={best_score[ct]:.4f}")

# 6. RE-TRAIN & FORECAST BOTH MODELS
# prepare test sequences
full_scaled = scaler.transform(df['population'].values.reshape(-1,1))[:,0]
inputs = full_scaled[-(TEST_SIZE + SEQ_LEN):]
x_test_seq = np.array([inputs[i-SEQ_LEN:i] for i in range(SEQ_LEN, len(inputs))])
x_test_seq = x_test_seq.reshape(-1, SEQ_LEN, 1)
y_test = test_df['population'].values
years_test = test_df['year'].values

predictions = {}
models = {}

for cell_type in ['LSTM','GRU']:
    params = best_params[cell_type]
    Cell = LSTM if cell_type=='LSTM' else GRU

    # build & train on full train set
    model = Sequential([
        Input(shape=(SEQ_LEN,1)),
        Cell(params['units'], return_sequences=True),
        Cell(params['units']),
        Dense(1)
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    print(f"\nTraining best {cell_type} model:")
    model.fit(
        x_train_seq, y_train_seq,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        verbose=1,
        shuffle=False
    )

    # forecast
    pred_scaled = model.predict(x_test_seq, verbose=0)
    pred = scaler.inverse_transform(pred_scaled.reshape(-1,1)).flatten()
    predictions[cell_type] = pred
    models[cell_type] = model

# 7. METRICS & COMPARISON TABLE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = []
for ct in ['LSTM','GRU']:
    y_pred = predictions[ct]
    metrics.append({
        'Model':  ct,
        'MSE':    mean_squared_error(y_test, y_pred),
        'RMSE':   rmse(y_test, y_pred),
        'MAE':    mean_absolute_error(y_test, y_pred),
        'MAPE (%)': mean_absolute_percentage_error(y_test, y_pred)*100,
    })

metrics_df = pd.DataFrame(metrics).set_index('Model').round(4)
print("\nTest-set performance:")
print(metrics_df)

# comparison table
comp_df = pd.DataFrame({
    'Year':          years_test,
    'Actual':        y_test,
    'LSTM Forecast': predictions['LSTM'],
    'GRU Forecast':  predictions['GRU']
})
print("\nForecast comparison:")
print(comp_df.to_markdown(index=False))

# =======================
# 8. PLOT TEST FORECASTS
# =======================
plt.figure(figsize=(8,5))
plt.plot(years_test, y_test,               label='Actual',        color='black', linewidth=2)
plt.plot(years_test, predictions['LSTM'],  label='LSTM Forecast', linestyle='--', linewidth=2)
plt.plot(years_test, predictions['GRU'],   label='GRU Forecast',  linestyle=':',  linewidth=2)
plt.title("Test-Sample: Actual vs. LSTM & GRU Forecasts")
plt.xlabel("Year"); plt.ylabel("Population")
plt.legend(); plt.grid(True)
plt.show()
